In [1]:
# !pip install litellm -U -q

In [1]:
import litellm

import sys
sys.path.append("..")
from ragaai_catalyst.prompt_manager import PromptManager
from ragaai_catalyst import RagaAICatalyst

/Users/siddharthakosti/anaconda3/envs/raga_ai_catalyst/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


In [2]:
import dotenv
dotenv.load_dotenv()
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
catalyst = RagaAICatalyst(
    access_key="9Dr41iRXo4RhGLlLjnmq",
    secret_key="a7vEHZJrSdsZzkinTnXKZ3XNllzH0vu2EdooTNf5",
    base_url="https://llm-dev5.ragaai.ai/api"
)

Token(s) set successfully


In [4]:
prompt_manager = PromptManager("docker_auto_test")


### Non-hallucination example

In [5]:
# Get the prompt
prompt = prompt_manager.get_prompt("hallu", version='v3')

In [6]:
# Get prompt variables
prompt_variables = prompt.get_variables()
print("prompt_variables: ",prompt_variables)

prompt_variables:  ['context', 'llm_response', 'query']


In [7]:
# Get parameters
parameters = prompt.get_parameters()
print("parameters: ",parameters)

parameters:  {'frequency_penalty': 0.0, 'max_tokens': 200, 'presence_penalty': 0.0, 'temperature': 0.5, 'top_p': 1.0, 'model': 'gpt-3.5-turbo'}


In [8]:
# Compile the prompt
compiled_prompt = prompt.compile(
    query='What is the capital of France?',
    llm_response="The capital of France is Paris.",
    context="France is a country in Western Europe. Its capital and largest city is Paris."
)
print(compiled_prompt)

[{'content': 'You are an LLM evaluation tool. Given any Prompt, Context and Response you identify information and metrics such as Hallucination in response. You self check at least 5 times to make sure you are correct', 'role': 'system'}, {'content': 'This is the context: France is a country in Western Europe. Its capital and largest city is Paris.\n           This is LLM Response: The capital of France is Paris.\n           This is user query: What is the capital of France?\n\n           You have to check if the LLM Response is hallucinating wrt to user query and context.\n           LLM Response should be based on facts present only in Context.\n           New claims, wrong information, made-up information, or typical correct answers but not present in context should be called hallucination.\n           Verify 5 times before responding.\n           Answer in True if it is hallucinationg or False otherwise and give descriptive reason why you believe this is the answer.\n           Als

In [9]:
# Function to call LiteLLM API
def get_litellm_response(prompt):
    response = litellm.completion(
        model="gpt-4o-mini",
        messages=prompt
    )
    return response.choices[0].message.content

In [10]:
# Get response from LiteLLM
litellm_response = get_litellm_response(compiled_prompt)

print("LiteLLM Response:")
print(litellm_response)

08:31:38 - LiteLLM:INFO: utils.py:2977 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
08:31:39 - LiteLLM:INFO: utils.py:1022 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


LiteLLM Response:
{"answer": "False", "reason": "The LLM Response accurately reflects the information present in the Context, which states that the capital of France is Paris. There are no new claims or incorrect information being presented.","reason_type": "correct response wrt query"}


### Hallucination example

In [11]:
# Compile the prompt
compiled_prompt = prompt.compile(
    query='What is the capital of France?',
    llm_response="The capital of France is Lyon.",
    context="France is a country in Western Europe. Its capital and largest city is Paris."
)
print(compiled_prompt)

[{'content': 'You are an LLM evaluation tool. Given any Prompt, Context and Response you identify information and metrics such as Hallucination in response. You self check at least 5 times to make sure you are correct', 'role': 'system'}, {'content': 'This is the context: France is a country in Western Europe. Its capital and largest city is Paris.\n           This is LLM Response: The capital of France is Lyon.\n           This is user query: What is the capital of France?\n\n           You have to check if the LLM Response is hallucinating wrt to user query and context.\n           LLM Response should be based on facts present only in Context.\n           New claims, wrong information, made-up information, or typical correct answers but not present in context should be called hallucination.\n           Verify 5 times before responding.\n           Answer in True if it is hallucinationg or False otherwise and give descriptive reason why you believe this is the answer.\n           Also

In [12]:
# Get response from LiteLLM
litellm_response = get_litellm_response(compiled_prompt)

print("LiteLLM Response:")
print(litellm_response)

08:32:19 - LiteLLM:INFO: utils.py:2977 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
08:32:20 - LiteLLM:INFO: utils.py:1022 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


LiteLLM Response:
{"answer": "True", "reason": "The response incorrectly states that the capital of France is Lyon, while the context explicitly states that the capital is Paris.","reason_type": "Contradictory wrt context"}
